In [1]:
cookies = {
    "contentPopup": "false",
    "KGID": "037b1c98-8e35-41d1-a270-e040b2c9ec7e",
    "fp_token_7c6a6574-f011-4c9a-abdd-9894a102ccef": "jLgiO7EaKyDH6WwVRfn6fPuB7r3m74SKpmxjNftYpMs=",
    "jwtLocalStorage": "1",
    "userPreferredContent": "1p",
    "KGID": "88138f05-3a22-5bd3-b5c2-b10afc8b6696",
    "MSG_LEG_TKN": "+DEZQ0sdLj20UcyFUPUrgg==",
    "dataSectionTemp": "0",
    "PHPSESSID": "MM9b8c187c47d8e042f16c78771be5c4fc1cdf24",
    "API_KEY": "L%2B%2F7AuhlpManHlqaewByIvfDq8DbmadUKo%2FVHdGIOc%2FWsx9PVVz8eO%2BOFZA%2Fxygi",
    "timezone": "Europe%2FAthens",
    "mvAnnouncement": "pNl7QkKHLQIt",
    "AWSALB": "JAcplMdgqSovZ2NAX3Q+z2Ubpn3QaqJHAYUeVIzsNhsXHoiNZQuSehWZXS1tOlmJH/4DRgi0NRZntM2PhjuvMUWulzkUHBPockTRloYjUV9kh3qHQUYMkie/JqRK",
    "AWSALBCORS": "JAcplMdgqSovZ2NAX3Q+z2Ubpn3QaqJHAYUeVIzsNhsXHoiNZQuSehWZXS1tOlmJH/4DRgi0NRZntM2PhjuvMUWulzkUHBPockTRloYjUV9kh3qHQUYMkie/JqRK",
    "XSRF-TOKEN": "eyJpdiI6ImRZRjRJbnVYQVRVWjgyVFoyWjlOT0E9PSIsInZhbHVlIjoiZEFpXC91b3Jzbmx2cVR2K0VPQlZkbTlmSUZcL3MrTDlaZ0JQdFVLRTNWeUM2eUhCWjA5UWMyVEdZeCtiekdCcm9PIiwibWFjIjoiNjE5Yzc0NWQwZWRiZjVkOTg0OGM2NmRkNzY3OWQzZDQ2NzA5OWRmZjdhZjJkY2UyODQ3NTk2YmMzMjBkNjZhZiJ9",
    "aws-waf-token": "9426a3d5-5342-463d-b871-3d87a01cd444:DQoAjfoODVkSAAAA:tbHIGPV4J3kIcXyjK5m3o4NG0GAxD0oV5G/Eum90/elbpo4zkSJNBJdkAUgX7sBXEnY6lEmdXkGhtPyNnPBaScAyw0en0BAzvfyghhj8vtwANsPh3fx5AYIjx3IXC+Ox3mRhgObZK6lXbnN9rSd7ZjhC4Lqy52SeYAvphdTT15dXbwbKLcCn+SE7RO6bCLJNYZFyvpVG9zfIXc2CKWfrYPxAY03NNC0s6Z/97xAZqa2sweOxDJ/tGL04dvPOv0fSOpJI",
}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0",
    "authority": "www.manyvids.com",
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9,fi;q=0.8",
    "cache-control": "no-cache",
    "pragma": "no-cache",
    "referer": "https://www.manyvids.com/Profile/354029/ellie-idol/Store/Videos?bundle=true&sort=newest",
    "sec-ch-ua": '"Chromium";v="130", "Microsoft Edge";v="130", "Not?A_Brand";v="99"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
}

In [2]:
# Define base path and filenames
base_path = r"Y:\Ellie Idol Rip"

In [ ]:
import requests
import json
import time
from tqdm import tqdm

# Set up the session and headers
session = requests.Session()
session.headers.update(headers)

for cookie_name, cookie_value in cookies.items():
    session.cookies.set(cookie_name, cookie_value, domain="www.manyvids.com")

# Download function
def download_page(page_number):
    url = f"https://www.manyvids.com/bff/store/videos/354029/?bundle=true&sort=newest&page={page_number}"
    response = session.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to download page {page_number}. Status code: {response.status_code}")
        return None

# Download 91 pages
total_pages = 91
for page in tqdm(range(1, total_pages + 1), desc="Downloading pages"):
    data = download_page(page)
    if data:
        with open(f"manyvids_page_{page:03d}.json", "w") as f:
            json.dump(data, f)
    time.sleep(1)  # Add a delay to avoid overwhelming the server

print("Download complete.")


In [ ]:
import pandas as pd
import json
import glob
from datetime import datetime

# Function to parse duration string to seconds
def parse_duration(duration_str):
    try:
        time = datetime.strptime(duration_str, "%M:%S")
        return time.minute * 60 + time.second
    except ValueError:
        return None

# List all JSON files
json_files = glob.glob("manyvids_page_*.json")

# Initialize an empty list to store all releases
all_releases = []

# Read each JSON file and extract the required data
for file in json_files:
    with open(file, 'r') as f:
        data = json.load(f)
        releases = data.get('data', [])
        for release in releases:
            # Extract URL directly from thumbnail and preview
            if 'thumbnail' in release and isinstance(release['thumbnail'], dict):
                release['thumbnail'] = release['thumbnail'].get('url', '')
            if 'preview' in release and isinstance(release['preview'], dict):
                release['preview'] = release['preview'].get('url', '')
            all_releases.append(release)

# Create a DataFrame from the extracted data
df = pd.DataFrame(all_releases)

# Parse duration column
df['duration'] = df['duration'].apply(parse_duration)

# Add URL column
df['url'] = df.apply(lambda row: f"https://www.manyvids.com/Video/{row['id']}/{row['slug']}", axis=1)

# Display the first few rows and basic information about the DataFrame
print(df.head())
print(df.info())


In [ ]:
import newnewid
import os
import re

# Function to extract ID and UUID from filename
def extract_id_uuid(filename):
    match = re.search(r'(\d{7}) - .+ - ([0-9a-f-]+)\.json$', filename)
    if match:
        return match.group(1), match.group(2)
    return None, None

# Dictionary to store existing ID-UUID mappings
existing_uuids = {}

# Load existing UUIDs from the folder
folder_path = r"Y:\Ellie Idol Rip"
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        id, uuid = extract_id_uuid(filename)
        if id and uuid:
            existing_uuids[id] = uuid
            print(f"Loaded {id} - {uuid}")

# Create a new UUID column
df['uuid'] = df['id'].apply(lambda x: existing_uuids.get(str(x).zfill(7), str(newnewid.uuid7())))

# Display the first few rows to verify the new column
print(df[['id', 'uuid']].head())


In [ ]:
# Write DataFrame to JSON file
output_file = os.path.join(base_path, 'manyvids_data.json')
df.to_json(output_file, orient='records', indent=2)
print(f"DataFrame has been written to {output_file}")


In [5]:
import os
import pandas as pd

output_file = os.path.join(base_path, 'manyvids_data.json')
df = pd.read_json(output_file)
print(df.head())


        id                                              title  \
0  5960560                      IT'S BUSH SEASON - ELLIE IDOL   
1  5708536          YOUR ONLY VIRTUAL GIRLFRIEND - ELLIE IDOL   
2  5960895  SUMMER ENDS AND THE DILF CAN FINALLY WANK 1080...   
3  5735011         CUCKING MY BOYFRIEND WITH YOU - ELLIE IDOL   
4  5783070         DEMONESS OFFERS A BI PARADISE - ELLIE IDOL   

                                                slug  duration  \
0                         its-bush-season-ellie-idol     530.0   
1            your-only-virtual-girlfriend-ellie-idol    1705.0   
2  summer-ends-and-the-dilf-can-finally-wank-1080...     524.0   
3           cucking-my-boyfriend-with-you-ellie-idol     686.0   
4           demoness-offers-a-bi-paradise-ellie-idol     728.0   

                                             creator  \
0  {'id': '354029', 'slug': 'Ellie-Idol', 'stageN...   
1  {'id': '354029', 'slug': 'Ellie-Idol', 'stageN...   
2  {'id': '354029', 'slug': 'Ellie-Idol', 's

In [ ]:
import requests
import json

# URL for the test request
url = "https://www.manyvids.com/bff/store/video/2652645/private"

# Make the request
response = requests.get(url, headers=headers, cookies=cookies)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    
    # Pretty print the JSON data
    print(json.dumps(data, indent=2))
else:
    print(f"Request failed with status code: {response.status_code}")
    print(response.text)


In [ ]:
import os
from tqdm import tqdm

for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing videos"):
    id = row['id']
    uuid = row['uuid']

    video_metadata_url = f"https://www.manyvids.com/bff/store/video/{id}"
    video_url = f"https://www.manyvids.com/bff/store/video/{id}/private"
    
    # Check if metadata file exists
    video_metadata_response = requests.get(video_metadata_url, headers=headers, cookies=cookies)
    if video_metadata_response.status_code == 200:
        date_raw = video_metadata_response.json()['data']['launchDate']
        date_obj = datetime.fromisoformat(date_raw)
        formatted_date = date_obj.strftime("%Y-%m-%d")
        
        title = video_metadata_response.json()['data']['title']
        # Sanitize the title for Windows filenames
        invalid_chars = r'<>:"/\|?*'
        clean_title = ''.join(c for c in title if c not in invalid_chars)
        clean_title = clean_title.strip()  # Remove leading/trailing whitespace
    
        id = video_metadata_response.json()['data']['id']
        
        base_filename = f'Ellie Idol - {formatted_date} - {id} - {clean_title} - {uuid}'
        json_filename = f'{base_filename}.json'
        json_full_path = os.path.join(base_path, json_filename)
        
        video_filename = f'{base_filename}.mp4'
        video_full_path = os.path.join(base_path, video_filename)

        thumbnail_filename = f'{base_filename}.jpg'
        thumbnail_full_path = os.path.join(base_path, thumbnail_filename)

        preview_filename = f'{base_filename} - Preview.mp4'
        preview_full_path = os.path.join(base_path, preview_filename)

        screenshot_filename = f'{base_filename} - Screenshot.jpg'
        screenshot_full_path = os.path.join(base_path, screenshot_filename)

        # Check if JSON file already exists
        if not os.path.exists(json_full_path):
            with open(json_full_path, 'w') as json_file:
                json.dump(video_metadata_response.json(), json_file, indent=2)
            tqdm.write(f"Metadata saved as {json_filename}")
        else:
            tqdm.write(f"Metadata file {json_filename} already exists. Skipping.")

        # Check if video file already exists
        if not os.path.exists(video_full_path):
            video_response = requests.get(video_url, headers=headers, cookies=cookies)
            if video_response.status_code == 200:
                video_data = video_response.json()
                video_file_url = video_data.get('data', {}).get('filepath')
                
                if video_file_url:
                    video_response = requests.get(video_file_url, headers=headers, cookies=cookies, stream=True)
                    
                    if video_response.status_code == 200:
                        os.makedirs(base_path, exist_ok=True)
                        
                        total_size = int(video_response.headers.get('content-length', 0))
                        with open(video_full_path, 'wb') as f, tqdm(
                            desc=video_filename,
                            total=total_size,
                            unit='iB',
                            unit_scale=True,
                            unit_divisor=1024,
                        ) as progress_bar:
                            for chunk in video_response.iter_content(chunk_size=8192):
                                size = f.write(chunk)
                                progress_bar.update(size)
                        
                        tqdm.write(f"Video downloaded and saved as {video_filename}")
                    else:
                        tqdm.write(f"Failed to download video. Status code: {video_response.status_code}")
                else:
                    tqdm.write("Video file URL not found in the response")
            else:
                tqdm.write(f"Request failed with status code: {video_response.status_code}")
                tqdm.write(video_response.text)
        else:
            tqdm.write(f"Video file {video_filename} already exists. Skipping.")

        # Download thumbnail
        if not os.path.exists(thumbnail_full_path):
            thumbnail_url = row['thumbnail']
            if thumbnail_url and not pd.isna(thumbnail_url):
                thumbnail_response = requests.get(thumbnail_url, headers=headers, cookies=cookies)
                if thumbnail_response.status_code == 200:
                    with open(thumbnail_full_path, 'wb') as f:
                        f.write(thumbnail_response.content)
                    tqdm.write(f"Thumbnail downloaded and saved as {thumbnail_filename}")
                else:
                    tqdm.write(f"Failed to download thumbnail. Status code: {thumbnail_response.status_code}")
            else:
                tqdm.write("Thumbnail URL not found or is NaN. Skipping.")
        else:
            tqdm.write(f"Thumbnail file {thumbnail_filename} already exists. Skipping.")

        # Download screenshot
        screenshot_url = video_metadata_response.json()['data'].get('screenshot')
        if screenshot_url and not os.path.exists(screenshot_full_path):
            screenshot_response = requests.get(screenshot_url, headers=headers, cookies=cookies)
            if screenshot_response.status_code == 200:
                with open(screenshot_full_path, 'wb') as f:
                    f.write(screenshot_response.content)
                tqdm.write(f"Screenshot downloaded and saved as {screenshot_filename}")
            else:
                tqdm.write(f"Failed to download screenshot. Status code: {screenshot_response.status_code}")
        elif not screenshot_url:
            tqdm.write("Screenshot URL not found in the response. Skipping.")
        else:
            tqdm.write(f"Screenshot file {screenshot_filename} already exists. Skipping.")

        # Download preview video
        preview_url = row.get('preview')
        if preview_url and not pd.isna(preview_url) and not os.path.exists(preview_full_path):
            preview_response = requests.get(preview_url, headers=headers, cookies=cookies, stream=True)
            if preview_response.status_code == 200:
                total_size = int(preview_response.headers.get('content-length', 0))
                with open(preview_full_path, 'wb') as f, tqdm(
                    desc=preview_filename,
                    total=total_size,
                    unit='iB',
                    unit_scale=True,
                    unit_divisor=1024,
                ) as progress_bar:
                    for chunk in preview_response.iter_content(chunk_size=8192):
                        size = f.write(chunk)
                        progress_bar.update(size)
                tqdm.write(f"Preview video downloaded and saved as {preview_filename}")
            else:
                tqdm.write(f"Failed to download preview video. Status code: {preview_response.status_code}")
        elif not preview_url or pd.isna(preview_url):
            tqdm.write("Preview video URL not found or is NaN. Skipping.")
        else:
            tqdm.write(f"Preview video file {preview_filename} already exists. Skipping.")
    else:
        tqdm.write(f"Failed to fetch metadata. Status code: {video_metadata_response.status_code}")
        tqdm.write(video_metadata_response.text)


In [ ]:
df[df['uuid'] == '019247b8-5a38-73ef-8aa1-07d165d4b620']


In [10]:
import os
import json
import subprocess
from tqdm import tqdm

# Function to run videohashes command and return JSON result
def get_video_hash(file_path):
    try:
        result = subprocess.run([r"C:\Tools\videohashes-windows-amd64.exe", "-md5", "-json", file_path], 
                                capture_output=True, text=True, check=True)
        return json.loads(result.stdout)
    except subprocess.CalledProcessError as e:
        tqdm.write(f"Error processing {file_path}: {e}")
        return None
    except json.JSONDecodeError as e:
        tqdm.write(f"Error decoding JSON for {file_path}: {e}")
        tqdm.write(f"Invalid JSON value: {result.stdout}")
        return None

# Get all video files in the base_path directory
video_files = [f for f in os.listdir(base_path) if f.endswith('.mp4') and not f.endswith('Preview.mp4')]

# Process each video file
for video_file in tqdm(video_files, desc="Processing videos"):
    file_path = os.path.join(base_path, video_file)
    
    # Try to find the UUID in the filename
    parts = video_file.split(' - ')
    uuid = None
    if len(parts) >= 5:
        potential_uuid = parts[-1].split('.')[0].strip()
        if len(potential_uuid) == 36:  # Assuming UUID is 36 characters long
            uuid = potential_uuid
    
    if uuid and uuid in df['uuid'].values:
        # Check if any of the hash columns are not yet populated
        row = df.loc[df['uuid'] == uuid].iloc[0]
        if pd.isna(row['duration']) or pd.isna(row['phash']) or pd.isna(row['oshash']) or pd.isna(row['md5']):
            hash_result = get_video_hash(file_path)
            if hash_result:
                df.loc[df['uuid'] == uuid, ['duration', 'phash', 'oshash', 'md5']] = [
                    hash_result.get('duration'),
                    hash_result.get('phash'),
                    hash_result.get('oshash'),
                    hash_result.get('md5')
                ]
        else:
            tqdm.write(f"Hash data already exists for file: {video_file}. Skipping.")
    else:
        tqdm.write(f"UUID not found or not in DataFrame for file: {video_file}. Skipping.")

hashes_output_file = os.path.join(base_path, 'manyvids_data_hashes.json')
df.to_json(hashes_output_file, orient='records', indent=2)
print(f"DataFrame has been written to {hashes_output_file}")


Processing videos:   5%|▌         | 42/812 [00:00<00:01, 413.99it/s]

Hash data already exists for file: Ellie Idol - 2015-09-02 - 80465 - CUM 3 TIMES IN 11 MINUTES WITH CEI - 019247b8-5a38-73ef-8aa1-07d165d4b620.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2015-09-05 - 81176 - POWER PLAY 720P - 019247b8-5a38-73ee-8d41-bc8a9ca54a03.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2015-09-07 - 82040 - 2 SCHOOL GIRLS COCK TEASE PRINCIPAL - 019247b8-5a38-73ed-81a8-8d33310b9264.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2015-09-07 - 82063 - MANDY THE TROPHY WIFE - 019247b8-5a38-73ec-af57-03667ee51698.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2015-09-07 - 82088 - WITH JOHN MUTUAL MASTURBATION - 019247b8-5a38-73eb-9de9-44579cd35d28.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2015-09-07 - 82108 - CUM WITH ME ROB - 019247b8-5a38-73ea-8863-16142c074acd.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2015-09-07 - 82321 - MANDYS IMPOSED ORGASM - 019247b8-5a38-

Processing videos:  17%|█▋        | 142/812 [00:00<00:01, 479.41it/s]

Hash data already exists for file: Ellie Idol - 2016-06-07 - 204198 - HARLEY QUINN'S IMPOSED BI FUNHOUSE - 019247b8-5a38-739c-bd63-0a2014038909.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2016-06-15 - 215175 - VR360 LITTLE STUDIO HELPER GETS A TREAT - 019247b8-5a38-739b-be63-d818993ea535.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2016-06-18 - 216233 - SECURING MY POSITION - 019247b8-5a38-739a-b372-ad0bf395b4d7.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2016-06-20 - 215297 - GOING FOR DISTANCE - 019247b8-5a38-7399-bcca-12435bec8fec.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2016-06-29 - 222736 - SNIFF MY GODDESS PUSSY POPS PLAY - 019247b8-5a38-7398-8753-dd36492bc253.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2016-06-30 - 223760 - CUM, EAT IT OR RUIN IT JOI GAME - 019247b8-5a38-7397-b964-8e33b8eca966.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2016-07-01 - 224337 - FATHER

Processing videos:  29%|██▉       | 238/812 [00:00<00:01, 455.76it/s]

...

Hash data already exists for file: Ellie Idol - 2020-11-26 - 2257004 - PUCKERED SAILOR MOON LIPS - 019245f6-9a46-7724-b8f8-a149d800df41.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2020-12-02 - 2344363 - TOUCH TO TITS WHILE I TOUCH MYSELF - 019245f6-9a46-7723-b68b-9fe12ca43bb9.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2020-12-08 - 2344354 - DIRTY LITTLE CUM EATER FOR TITTIES - 019245f6-9a46-7722-b177-ffb4ca8f4ed4.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2020-12-11 - 2344332 - CELL PHONE FARTS IN LOCKDOWN - 019245f6-9a46-7721-893a-c1e524b47e6c.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2020-12-14 - 2344324 - FATAL TOILET SLAVERY FOR TWO - 019245f6-9a46-7720-a34b-7c87618bc6bd.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2020-12-17 - 2352538 - THE WEDDING NIGHT AFFAIR - 019245f6-9a46-771f-8e76-753e2f756ce9.mp4. Skipping.
Hash data already exists for file: Ellie Idol - 2020-12-20 - 2414917 - 

Processing videos:  98%|█████████▊| 799/812 [1:00:21<01:33,  7.19s/it]

Error decoding JSON for Y:\Ellie Idol Rip\Ellie Idol - 2024-08-04 - 5720276 - MY ROOMMATE IS TOO HORNY AND NEEDS CHASTITY BUT FIRST - 019245f6-9a3d-7397-a6a6-ac906306cc4a.mp4: Expecting value: line 1 column 1 (char 0)
Invalid JSON value: error reading video file: FFProbe encountered an error with <Y:\Ellie Idol Rip\Ellie Idol - 2024-08-04 - 5720276 - MY ROOMMATE IS TOO HORNY AND NEEDS CHASTITY BUT FIRST - 019245f6-9a3d-7397-a6a6-ac906306cc4a.mp4>.
Error JSON:
{
    "error": {
        "code": -1094995529,
        "string": "Invalid data found when processing input"
    }
}

Error: exit status 1



Processing videos: 100%|██████████| 812/812 [1:02:49<00:00,  4.64s/it]

DataFrame has been written to Y:\Ellie Idol Rip\manyvids_data_hashes.json


In [11]:
hashes_output_file = os.path.join(base_path, 'manyvids_data_hashes.json')
df.to_json(hashes_output_file, orient='records', indent=2)
print(f"DataFrame has been written to {hashes_output_file}")

DataFrame has been written to Y:\Ellie Idol Rip\manyvids_data_hashes.json


In [16]:
import json
from pathlib import Path
from tqdm import tqdm

# Create a new DataFrame to store the JSON data
json_df = pd.DataFrame(columns=['id', 'title', 'videoDuration', 'description', 'tagList', 'filepath'])

# Iterate through all MP4 files in the base_path
for video_file in tqdm(Path(base_path).glob('*.mp4'), desc="Processing JSON files"):
    # Construct the corresponding JSON filename
    json_file = video_file.with_suffix('.json')
    
    # Check if the JSON file exists
    if json_file.exists():
        try:
            with open(json_file, 'r') as f:
                json_data = json.load(f)
            
            # Extract the relevant data from the JSON
            data = json_data.get('data', {})
            tag_list = data.get('tagList', [])
            
            # Extract only the 'label' from each tag
            tag_labels = [tag['label'] for tag in tag_list if 'label' in tag]
            
            row_data = {
                'id': data.get('id'),
                'title': data.get('title'),
                'videoDuration': data.get('videoDuration'),
                'description': data.get('description'),
                'tagList': ', '.join(tag_labels),  # Join the tag labels
                'filepath': str(video_file)
            }
            
            # Append the data to the DataFrame
            json_df = pd.concat([json_df, pd.DataFrame([row_data])], ignore_index=True)
        
        except json.JSONDecodeError:
            tqdm.write(f"Error reading JSON file: {json_file}")
        except Exception as e:
            tqdm.write(f"Error processing file {json_file}: {str(e)}")

print(f"Processed {len(json_df)} JSON files.")
print(json_df.head())

Processing JSON files: 1386it [00:00, 2868.74it/s]

Processed 811 JSON files.
      id                                title videoDuration  \
0  80465   CUM 3 TIMES IN 11 MINUTES WITH CEI         11:43   
1  81176                      POWER PLAY 720P         12:13   
2  82040  2 SCHOOL GIRLS COCK TEASE PRINCIPAL         18:05   
3  82063                MANDY THE TROPHY WIFE         06:14   
4  82088        WITH JOHN MUTUAL MASTURBATION         06:50   

                                         description  \
0  I will let you stroke it to my juicy booty! Je...   
1  Mandy and I are hanging out on the couch. I'm ...   
2  Mandy and I are in the principal's office once...   
3  Mandy invites me over to meet her husband. He ...   
4  I know you'd love nothing more than for me to ...   

                                             tagList  \
0  Ass, AssWorship, CumEatingInstruction, Imposed...   
1  ElectricPlay, Hitachi, Squirt, Squirting, ToeF...   
2  CockTease, Lesbians, PussyEating, SchoolGirl, ...   
3  BratGirls, CockTease, Cuckoldin

# Stash import

In [12]:
# Create Stash app client

import pandas as pd
import dotenv

from libraries.client_stashapp import get_stashapp_client

dotenv.load_dotenv()

stash = get_stashapp_client()

dUsing stash (v0.27.0-0) endpoint at http://localhost:6969/graphql
dPersisting Connection to Stash with ApiKey...


In [19]:
all_stash_tags = stash.find_tags()
df_all_stash_tags = pd.DataFrame(all_stash_tags)
print(df_all_stash_tags.head())


     id                  name  \
0  5045          2D Available   
1  5049          3D Available   
2  5050          3K Available   
3  5051  3rd Person Narrative   
4  5053      4:3 Aspect Ratio   

                                         description  \
0  3D or VR scenes that offer a more traditional ...   
1  Offered in a format with a three-dimensional e...   
2     Scenes offered in a resolution 3K pixels wide.   
3  Features a storyline with fictional characters...   
4       Footage shot in a 4:3 (1.33:1) aspect ratio.   

                                             aliases  ignore_auto_tag  \
0               [2-D, 2D, 2D Video, Two-Dimensional]            False   
1             [3-D, 3D, 3D Video, Three-Dimensional]            False   
2              [1600p, 3K VP9, 3K VR, 3K h264, 3KVR]            False   
3  [3rd Person Perspective, Third Person Perspect...            False   
4  [1.33:1, 1.33:1 Aspect Ratio, 1.33x1, 4:3, 4x3...            False   

             created_at  

In [20]:
import json
from pathlib import Path
from tqdm import tqdm

df_manyvids_tags = pd.DataFrame(columns=['id', 'label'])

# Iterate through all MP4 files in the base_path
for video_file in tqdm(Path(base_path).glob('*.mp4'), desc="Processing JSON files"):
    # Construct the corresponding JSON filename
    json_file = video_file.with_suffix('.json')
    
    # Check if the JSON file exists
    if json_file.exists():
        try:
            with open(json_file, 'r') as f:
                json_data = json.load(f)
            
            # Create a temporary DataFrame for new tags
            new_tags = pd.DataFrame(json_data['data']['tagList'])
            
            # Add new tags to df_manyvids_tags if they don't already exist
            df_manyvids_tags = pd.concat([df_manyvids_tags, new_tags[~new_tags['id'].isin(df_manyvids_tags['id'])]], ignore_index=True)
        
        except json.JSONDecodeError:
            tqdm.write(f"Error reading JSON file: {json_file}")
        except Exception as e:
            tqdm.write(f"Error processing file {json_file}: {str(e)}")

print(f"Processed {len(df_manyvids_tags)} unique tags.")
print(df_manyvids_tags.head())

Processing JSON files: 1386it [00:00, 1715.53it/s]

Processed 515 unique tags.
    id                 label  count
0    4                   Ass    1.0
1  165            AssWorship    1.0
2  339  CumEatingInstruction    1.0
3  471   ImposedEjaculations    1.0
4  586    JerkOffInstruction    1.0


In [21]:
# Function to add spaces to tag names
def add_spaces_to_tag_name(tag_name):
    # Use regex to split the string at the boundary between lowercase and uppercase letters
    import re
    return re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', tag_name)

# Apply the function to the 'label' column and add it as a new column 'clean_label'
df_manyvids_tags['clean_label'] = df_manyvids_tags['label'].apply(add_spaces_to_tag_name)

# Print the first few rows to verify the changes
print(df_manyvids_tags[['label', 'clean_label']].head())

# Create a dictionary mapping ManyVids tag IDs to their corrected names
manyvids_tag_dict = dict(zip(df_manyvids_tags['id'], df_manyvids_tags['clean_label']))

print(f"Processed {len(manyvids_tag_dict)} unique tags.")


                  label             clean_label
0                   Ass                     Ass
1            AssWorship             Ass Worship
2  CumEatingInstruction  Cum Eating Instruction
3   ImposedEjaculations    Imposed Ejaculations
4    JerkOffInstruction    Jerk Off Instruction
Processed 515 unique tags.


In [28]:
# Import necessary libraries
from fuzzywuzzy import process

# Function to find the best match using Levenshtein distance
def find_best_match(manyvids_tag, stash_tags, threshold=95):
    best_match = process.extractOne(manyvids_tag, stash_tags)
    if best_match and best_match[1] >= threshold:
        return best_match[0]
    return None

# Create a dictionary to store the matches
tag_matches = {}

# Iterate through ManyVids tags and find matches in Stash tags
for mv_id, mv_tag in tqdm(manyvids_tag_dict.items(), desc="Matching tags"):
    best_match = find_best_match(mv_tag, df_all_stash_tags['name'])
    if best_match:
        stash_tag = df_all_stash_tags[df_all_stash_tags['name'] == best_match].iloc[0]
        tag_matches[mv_id] = {
            'manyvids_tag': mv_tag,
            'stash_tag_id': stash_tag['id'],
            'stash_tag_name': stash_tag['name']
        }
    else:
        tag_matches[mv_id] = {
            'manyvids_tag': mv_tag,
            'stash_tag_id': '',
            'stash_tag_name': ''
        }

# Create a DataFrame from the matches for easier viewing
df_tag_matches = pd.DataFrame.from_dict(tag_matches, orient='index')

print(f"Processed {len(df_tag_matches)} tags out of {len(manyvids_tag_dict)} ManyVids tags.")
print(f"Matched {len(df_tag_matches[df_tag_matches['stash_tag_id'] != ''])} tags.")
print(df_tag_matches.head())

# Save the matches to a CSV file for manual review if needed
df_tag_matches.to_csv('tag_matches.csv', index=False)
print("Tag matches saved to 'tag_matches.csv'")


Matching tags: 100%|██████████| 515/515 [04:05<00:00,  2.10it/s]

Processed 515 tags out of 515 ManyVids tags.
Matched 197 tags.
               manyvids_tag stash_tag_id          stash_tag_name
4                       Ass                                     
165             Ass Worship         5184             Ass Worship
339  Cum Eating Instruction         5590  Cum Eating Instruction
471    Imposed Ejaculations                                     
586    Jerk Off Instruction         6242    Jerk Off Instruction
Tag matches saved to 'tag_matches.csv'


In [ ]:
for index, row in tqdm(df.iterrows(), total=len(df), desc="Importing videos to Stash"):
    video_data = {
        "title": row['title'],
        "date": row['date'],
        "url": row['url'],
        "studio": {"name": "ManyVids"},
        "performers": [{"name": "Ellie Idol"}],
        "tags": [],
        "files": [{"path": row['filepath']}],

